In [ ]:
!pip install requests bs4

In [ ]:
# 爬取猫眼影片总票房排行榜
import requests
from bs4 import BeautifulSoup

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 Edg/108.0.1462.42"
}  # 将程序发出的请求伪装成浏览器
text = requests.get("https://piaofang.maoyan.com/rankings/year", headers=headers).text
# 通过requests请求到电影票房的网页
main_page = BeautifulSoup(text, "html.parser")  # 后面这个是html的解析器
# 使用BeautifulSoup对text进行解析
div = main_page.find("div", attrs={"id": "ranks-list"})
# 找到名为div且id为ranks-list的所有标签
with open("MovieData.csv", mode="w", encoding="utf-8", newline='') as f:  # 创建一个名为MovieData.csv的文件，放在变量f中，且可以在其后面直接追加文字
    f.write("排名,片名,票房(万元),平均票价,场均人次\n")  # 写入表头
    uls = div.find_all('ul')
    # 找到div元素中所有名为ul的元素，并存储在uls变量中
    for ul in uls:  # 对uls中的元素进行遍历
        lst1 = ul.find_all("li", attrs={"class": "col1"})  # 找到ul中所有类名为col1的Li元素，并储存在lst1
        lst2 = ul.find_all("li", attrs={"class": "col2"})  # 找到ul中所有类名为col2的Li元素，并储存在lst2
        lst3 = ul.find_all("li", attrs={"class": "col3"})  # 找到ul中所有类名为col3的Li元素，并储存在lst3
        lst4 = ul.find_all("li", attrs={"class": "col4"})  # 找到ul中所有类名为col4的Li元素，并储存在lst4
        row_data = []  # 用于存储当前行的数据
        for li in lst1:  # 对lst1中的元素进行遍历
            ps = li.find_all("p")  # 找到ps中所有p元素，并储存在ps
            for p in ps:  # 对ps中的元素进行遍历
                row_data.append(p.text.strip())  # 将p标签中的文本去除分隔符后添加到row_data中
        for li in lst2:  # 对lst2中的元素进行遍历
            row_data.append(li.text.strip())  # 将li标签中的文本去除分隔符后添加到row_data中
        for li in lst3:  # 对lst3中的元素进行遍历
            row_data.append(li.text.strip())  # 将li标签中的文本去除分隔符后添加到row_data中
        for li in lst4:  # 对lst4中的元素进行遍历
            row_data.append(li.text.strip())  # 将li标签中的文本去除分隔符后添加到row_data中
        f.write(",".join(row_data) + "\n")  # 使用join方法将row_data中的元素连接成一个字符串，并写入文件